# Image Processing Notebook

This notebook takes in the images processed from meteoblue, along with the respective highest and lowest temperature, in order to determine for each coordinate, what the air temperature is.

Input: Meteoblue image in `.jpg` format

Output: Coordinates and temperature, in a csv format containing the following tables

| Date   | Time   | Latitude | Longitude | Temperature |
| ------ | ------ | -------- | --------- | ----------- |
| string | string | float    | float     | float       |

### ⚠️ **CAUTION** ⚠️

**Not all coordinates are in Singapore! This output will also produce coordinates that are out of Singapore (i.e. Johor Bahru, Singaporean waters, other Singaporean islands). Keep this in mind when processing the models.**

Alternatively, a filter may also be implemented, depending on whether it is needed or not

### Technicalities

This program analyses pixel by pixel. It assumes a discrete coordinate system, in order to output sampled data. 

For coordinates whose colors are not standardised, the program will use **linear interpolation** to infer the closest temperature. It might not reflect the actual temperature, but should provide a good enough estimate considering a linear model.

Other interpolation methods may be explored, depending on availability of time

### Setup

Imports image and converts the image to a processable numpy array. Also initialises the dataframe to post the data into

In [19]:
# Import statements
from PIL import Image
import numpy as np
import pandas as pd

# Load image as numpy array
im = Image.open('singapore_2024040910.jpg')
im = im.convert('RGB')

# Convert image to numpy array
data = np.array(im)

# Initialise new pandas dataframe 
df = pd.DataFrame(columns=['Date', 'Time', 'Latitude', 'Longitude', 'Temperature'])

### Constant Declarations

- Sets the latitude and longitude bounds of the image
- Sets the image dimensions (image shape, assuming the image size and resolution is constant)
- Sets the color bounds

In [20]:
# Lat-long bounds
lat_min = 1.19966
lat_max = 1.47512
long_min = 103.59258
long_max = 104.01814

# Image shape
height = data.shape[0]
width = data.shape[1]

# Predetermined colors (meteoblue color scale)
colors = [
    "#563853",
    "#5C3959",
    "#763E70",
    "#94438C",
    "#B967B1",
    "#AF72BD",
    "#AA96D7",
    "#A8B8E4",
    "#A7CDE3",
    "#95DBD7",
    "#77CCC6",
    "#76C0CA",
    "#73ABC8",
    "#77A0D0",
    "#639CA7",
    "#5E9B90",
    "#6AA262",
    "#83A933",
    "#ABB033",
    "#CDB933",
    "#EDC233",
    "#FBAF33",
    "#F98A33",
    "#EE6933",
    "#B95233",
    "#974433",
    "#733933"
]

# Function to convert color hexadecimal into array of RGB values
def hex_to_rgb(hex):
    return [int(hex[i:i+2], 16) for i in (1, 3, 5)]

colors = [hex_to_rgb(color) for color in colors]

### Take in minimum and maximum temperature for that image

Requires connecting to the meteoblue network to obtain the min/max temp

**TODO [JSON]: Extract temperature values from html file yo**

In [21]:
# Date and time values
date = "2024-04-09"
time = "10:00"

# Minimum and maximum temperature values
temp_min = 25.07141175346661
temp_max = 33.46679033118587

In [22]:
# Assign temperature to colors
color_map = []

for i in range(len(colors)):
    temp = temp_min + ((temp_max - temp_min) / (len(colors) - 1)) * i
    obj = {
        'color': colors[i],
        'temp': temp
    }
    color_map.append(obj)

color_map

[{'color': [86, 56, 83], 'temp': 25.07141175346661},
 {'color': [92, 57, 89], 'temp': 25.394310929532736},
 {'color': [118, 62, 112], 'temp': 25.71721010559886},
 {'color': [148, 67, 140], 'temp': 26.040109281664986},
 {'color': [185, 103, 177], 'temp': 26.363008457731112},
 {'color': [175, 114, 189], 'temp': 26.685907633797235},
 {'color': [170, 150, 215], 'temp': 27.008806809863362},
 {'color': [168, 184, 228], 'temp': 27.33170598592949},
 {'color': [167, 205, 227], 'temp': 27.65460516199561},
 {'color': [149, 219, 215], 'temp': 27.97750433806174},
 {'color': [119, 204, 198], 'temp': 28.300403514127865},
 {'color': [118, 192, 202], 'temp': 28.623302690193988},
 {'color': [115, 171, 200], 'temp': 28.946201866260115},
 {'color': [119, 160, 208], 'temp': 29.26910104232624},
 {'color': [99, 156, 167], 'temp': 29.592000218392364},
 {'color': [94, 155, 144], 'temp': 29.91489939445849},
 {'color': [106, 162, 98], 'temp': 30.237798570524618},
 {'color': [131, 169, 51], 'temp': 30.56069774659

### Image Processing Stage

Reads the image pixel by pixel, and matches against the colors matching. Assigns the temperature to the closest color for now.

In [23]:
# Utility Functions 

def color_diff(color1, color2):
    return sum([abs(color1[i] - color2[i]) for i in range(3)])

def closest_color(color):
    min_diff = color_diff(color, color_map[0]['color'])
    min_color = color_map[0]
    min_temp = color_map[0]['temp']
    for i in range(1, len(color_map)):
        diff = color_diff(color, color_map[i]['color'])
        if diff < min_diff:
            min_diff = diff
            min_color = color_map[i]
            min_temp = color_map[i]['temp']
    return min_color, min_temp

In [27]:
# Iterate through image and assign temperature values
for i in range(height):
    for j in range(width):
        if i % 10 == 0 and j % 10 == 0: # Only sample every 4th pixel, cos it's too slow otherwise
            color = data[i][j]
            closest, temp = closest_color(color)
            lat = lat_min + (lat_max - lat_min) * i / height
            long = long_min + (long_max - long_min) * j / width
            new_df = {
                'Date': date,
                'Time': time,
                'Latitude': lat,
                'Longitude': long,
                'Temperature': temp
            }
            df.loc[len(df)] = new_df

In [28]:
df

,Date,Time,Latitude,Longitude,Temperature
0,2024-04-09,10:00,1.199660,103.592580,30.883597
1,2024-04-09,10:00,1.199660,103.592837,27.331706
2,2024-04-09,10:00,1.199660,103.593094,27.331706
3,2024-04-09,10:00,1.199660,103.593351,27.331706
4,2024-04-09,10:00,1.199660,103.593608,27.654605
...,...,...,...,...,...
237599,2024-04-09,10:00,1.474092,104.013000,27.977504
237600,2024-04-09,10:00,1.474092,104.014028,27.977504
237601,2024-04-09,10:00,1.474092,104.015056,27.977504
237602,2024-04-09,10:00,1.474092,104.016084,27.654605
